In [1]:
%matplotlib inline
import pandas as pd

In [2]:
participation = pd.read_csv("data/users_participating_events.csv")

In [5]:
participation.head()

,user_facebook_id,rsvp_status,event_facebook_id
0,201098027006232,attending,1606969096264134
1,334566996897353,attending,1606969096264134
2,354393944896674,attending,1606969096264134
3,1698350500494930,attending,1606969096264134
4,1775177359410467,attending,1606969096264134


In [3]:
participation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3667328 entries, 0 to 3667327
Data columns (total 3 columns):
user_facebook_id     int64
rsvp_status          object
event_facebook_id    int64
dtypes: int64(2), object(1)
memory usage: 83.9+ MB


In [6]:
attending =participation.loc[lambda df: df.rsvp_status == 'attending', ['user_facebook_id', 'event_facebook_id']]

In [7]:
attending.head()

,user_facebook_id,event_facebook_id
0,201098027006232,1606969096264134
1,334566996897353,1606969096264134
2,354393944896674,1606969096264134
3,1698350500494930,1606969096264134
4,1775177359410467,1606969096264134


In [ ]:
attending.groupby('event_facebook_id').size()

In [15]:
pairs_raw = attending.merge(attending, on='user_facebook_id')

In [16]:
pairs_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19728557 entries, 0 to 19728556
Data columns (total 3 columns):
user_facebook_id       int64
event_facebook_id_x    int64
event_facebook_id_y    int64
dtypes: int64(3)
memory usage: 602.1 MB


In [25]:
pairs = pairs_raw.groupby(['event_facebook_id_x', 'event_facebook_id_y']).size()# \
#  .reset_index().rename(columns={0: 'count'})

In [37]:
pairs.shape

(5546890,)

In [39]:
pairs.loc[lambda ser: ser > 1].shape

(2594099,)

In [27]:
pairs.loc[100668907024083]

event_facebook_id_y
100668907024083      201
100671793684638        5
100860333400630        2
101225040052676        2
101593436871236        3
101703520161849        1
101814433512143        1
102247963441256       19
102630626836363        2
102652476748675       10
104605033224828        2
106378716226104        3
107749699638345        7
108144682857071        3
108953062776313        2
110475745978177        1
110939082450793        1
111191022375338        2
111739462317052        1
111750369011610        6
113001615707339        1
113267952414691        2
113621508794502        1
115511725301568        3
116014188539229        2
116471765209738        1
116502542068147        2
117000445382634        3
117347168470716        4
117558868427195        1
                    ... 
1808623656024527       1
1810276502532100       1
1813911045506541       7
1814269335472327       1
1818782285001635       2
1819398601630749       2
1820147331533165       5
1820175368218581       3
18218

In [35]:
!mkdir data/processed

In [36]:
pairs.to_csv("data/processed/pair_count.csv")

In [55]:
events = pd.read_csv("data/events.csv", index_col='facebook_id')
events = events.loc[:, ['attending_count', 'facebook_page', 'name', 'place_name', 'owner_id', 'owner_name']]

In [56]:
events.sample().index[0]

804368059632145

In [61]:
# 1411752285747539

In [62]:
def show_related(event_id=None, limit=10):
    if event_id is None:
        event_id = events.sample().index[0]
    print(events.loc[event_id])
    related = pairs[event_id].sort_values(ascending=False).head(limit)
    return related.to_frame().join(events, how='inner')

In [72]:
show_related()

attending_count                                 71
facebook_page                      180154812118956
name               BEZPOŚREDNIO – wernisaż wystawy
place_name                     Galeria Szyb Wilson
owner_id                               1.80155e+14
owner_name                     Galeria Szyb Wilson
Name: 600609960017572, dtype: object


,0,attending_count,facebook_page,name,place_name,owner_id,owner_name
600609960017572,68,71,180154812118956,BEZPOŚREDNIO – wernisaż wystawy,Galeria Szyb Wilson,1.801548e+14,Galeria Szyb Wilson
351496941724472,13,314,128885767128773,Wystawa KOŃ TROJAŃSKI,Galeria Sztuki Współczesnej BWA w Katowicach,NaN,NaN
613746512102998,12,383,459134480771462,PÓŁNOC/POŁUDNIE - Wystawa,MALARSTWO - ASP KATOWICE PAINTING - ACADEMY OF...,4.591345e+14,MALARSTWO - ASP KATOWICE PAINTING - ACADEMY OF...
1433054166996596,12,2013,179603073902,Katowice Street Art Festival 2015,NaN,1.004969e+14,Katowice Street Art Festival
281244615328638,10,53,65815660372,Michalina Wawrzyczek-Klasik „Pomiędzy”,NaN,6.581566e+10,Rondo Sztuki
1044609425589608,10,2371,334195793031,Festiwal Otwarcia nowej siedziby Muzeum Śląski...,Muzeum Śląskie,3.341958e+11,Muzeum Śląskie
1035823059763244,10,6220,179603073902,Archive i Morcheeba na 150-lecie Katowic,NaN,1.796031e+11,Katowice Miasto Ogrodów
619151424824747,9,127,65815660372,"Dominika Kowynia ""Take Care""",NaN,6.581566e+10,Rondo Sztuki
859350914156160,9,1560,1616002318619223,"ARTUR ROJEK Dj Set, KUBA SOJKA live - JOHN LEM...",GALOP,1.616002e+15,GALOP
937781489615129,9,1739,179603073902,Ars Independent Festival 2015,"Katowice, Woiwodschaft Schlesien, Polen",1.830239e+14,Ars Independent


In [64]:
pairs2 = pairs.reset_index().rename(columns={0: 'count'})

In [65]:
pairs2.head()

,event_facebook_id_x,event_facebook_id_y,count
0,100668907024083,100668907024083,201
1,100668907024083,100671793684638,5
2,100668907024083,100860333400630,2
3,100668907024083,101225040052676,2
4,100668907024083,101593436871236,3


In [ ]:
pairs2.merge( left_on='event_facebook_id_x')

## To do

* per miejsce
* per organizacja

x

* conditional probability (done, sort of)
* PMI